# マルチセッション評価

このノートブックは、LLM をジャッジとして使用する拡張可能な LLM ベース評価フレームワークである Strands Evals を使用してエージェントセッションを評価します。各セッションについて、AgentCore Observability からトレースを取得し、評価メトリクスを実行し、ダッシュボード相関のために元のトレース ID で結果をログに記録します。

**このノートブックは2つの評価メトリクスをデモンストレーションします:**
- **OutputEvaluator**: 応答品質をスコアリング（関連性、正確性、完全性）
- **TrajectoryEvaluator**: ツール使用をスコアリング（選択、効率、シーケンス）

Strands Evals は、ほぼあらゆる評価タイプのカスタム評価メトリクスをサポートしています。フレームワークの力はルーブリックシステムにあります。基準を定義すれば、LLM がそれらを一貫して適用します。

**ワークフロー:**
1. 検出ノートブックからセッションを読み込み（またはカスタムセッション ID を提供）
2. 各セッション: トレースを取得、評価ケースを作成、評価メトリクスを実行
3. EMF 形式で結果を AgentCore にログ
4. サマリー統計を生成

**前提条件:** まずセッション検出ノートブックを実行するか、セッション ID のリストを準備してください。

## この位置づけ

これは**ノートブック2（オプションA）**です - 定義したカスタムルーブリックを使用してセッションを評価します。

![Notebook Workflow](images/notebook_workflow.svg)

## データフロー

評価パイプラインは AgentCore Observability トレースをスコア付き結果に変換します:

![Evaluation Pipeline](images/evaluation_pipeline.svg)

## セットアップ

Strands Evals 評価メトリクスと AgentCore Observability インタラクション用のユーティリティクラスを含む必要なモジュールをインポートします。設定は `config.py` から読み込まれます。

In [ ]:
import logging
import sys
from datetime import datetime, timedelta, timezone
from typing import List

sys.path.insert(0, ".")

from config import (
    AWS_REGION,
    AWS_ACCOUNT_ID,
    SOURCE_LOG_GROUP,
    EVAL_RESULTS_LOG_GROUP,
    LOOKBACK_HOURS,
    MAX_CASES_PER_SESSION,
    DISCOVERED_SESSIONS_PATH,
    RESULTS_JSON_PATH,
    EVALUATION_CONFIG_ID,
    setup_cloudwatch_environment,
)

from utils import (
    CloudWatchSessionMapper,
    ObservabilityClient,
    SessionDiscoveryResult,
    SessionInfo,
    send_evaluation_to_cloudwatch,
)

from strands_evals import Case, Experiment
from strands_evals.evaluators import OutputEvaluator, TrajectoryEvaluator
from strands_evals.types.trace import AgentInvocationSpan, ToolExecutionSpan

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

## 設定

CloudWatch メトリクス用の評価メトリクス名を定義します。これらの名前は AgentCore Observability ダッシュボードに表示され、`Custom.YourEvaluatorName` 規約に従う必要があります。`EVALUATION_CONFIG_ID` は `config.py` から読み込まれます。

In [ ]:
# Custom evaluator names for CloudWatch metrics (customize for your use case)
OUTPUT_EVALUATOR_NAME = "Custom.OutputEvaluator"
TRAJECTORY_EVALUATOR_NAME = "Custom.TrajectoryEvaluator"

## CloudWatch 環境

評価結果のログに必要な環境変数を設定します。OTEL リソース属性に `config.py` の `SERVICE_NAME` を使用します。

In [ ]:
setup_cloudwatch_environment()

## セッションの読み込み

検出ノートブックの JSON 出力からセッションを読み込みます。または、`USE_JSON_FILE = False` に設定し、特定のセッションのターゲット再評価のためにカスタムセッション ID を直接提供します。

In [ ]:
# Set to False to provide custom session IDs instead
USE_JSON_FILE = True

if USE_JSON_FILE:
    discovery_result = SessionDiscoveryResult.load_from_json(DISCOVERED_SESSIONS_PATH)
    sessions_to_process = discovery_result.sessions
else:
    # Provide custom session IDs here
    session_ids = [
        "your-session-id-here",
    ]
    sessions_to_process = [
        SessionInfo(
            session_id=sid,
            span_count=0,
            first_seen=datetime.now(timezone.utc),
            last_seen=datetime.now(timezone.utc),
            discovery_method="user_provided",
        )
        for sid in session_ids
    ]

print(f"Loaded {len(sessions_to_process)} sessions")

## 評価メトリクスのルーブリック

ルーブリックは評価基準を定義します。評価メトリクスはルーブリックとエージェントの出力を LLM に送信し、LLM がジャッジとして機能してスコア（0.0-1.0）と説明を返します。

**効果的なルーブリックの作成:**
- 良い品質と悪い品質を構成するものについて具体的に記述する
- スコアリングアンカーを含める（1.0 vs 0.5 vs 0.0 は何を意味するか？）
- エージェントのドメインに関連する測定可能な基準に焦点を当てる

以下のルーブリックをカスタマイズしてください。デフォルトのルーブリックは一般的な応答品質とツール使用パターンを評価します。

In [ ]:
output_rubric = """
Evaluate the agent's response based on:
1. Relevance: Does the response directly address the user's question?
2. Accuracy: Is the information factually correct?
3. Completeness: Does the response provide sufficient detail?

Score 0.0-1.0: 1.0=excellent, 0.5=adequate, 0.0=poor
"""

trajectory_rubric = """
Evaluate the agent's tool usage based on:
1. Tool Selection: Did the agent choose appropriate tools?
2. Efficiency: Were tools used without unnecessary calls?
3. Logical Sequence: Were tools used in a logical order?

Score 0.0-1.0: 1.0=optimal, 0.5=acceptable, 0.0=poor
"""

## ヘルパー関数

これらの関数は AgentCore Observability トレースと Strands Evals を橋渡しします:

- `task_fn(case)`: OutputEvaluator がルーブリックに対してスコアリングするためのエージェントの実際の応答を返します。

- `trajectory_task_fn(case)`: TrajectoryEvaluator がツール使用パターンを評価するための応答とツールシーケンスの両方を返します。

- `create_cases_from_session(session)`: Strands Eval Session を評価 Cases に変換します。AgentInvocationSpan からユーザープロンプトを抽出し、ToolExecutionSpan オブジェクトからツール名を抽出し、CloudWatch 相関用に元の trace_id を保持します。

- `log_case_result_to_cloudwatch(case, ...)`: 元の trace_id を使用して評価結果を AgentCore Observability に送信し、ダッシュボードで元のトレースと一緒にスコアを確認できるようにします。

In [ ]:
def task_fn(case: Case) -> str:
    """Return actual output from trace metadata."""
    return (case.metadata.get("actual_output", ""))


def trajectory_task_fn(case: Case):
    """Return output and trajectory from trace metadata."""
    return {"output": case.metadata.get("actual_output", ""), "trajectory": case.metadata.get("trajectory_for_eval", [])}

def log_case_result_to_cloudwatch(case: Case, evaluator_name: str, score: float, explanation: str, label: str = None) -> bool:
    """Log evaluation result to CloudWatch with original trace ID."""
    trace_id = case.metadata.get("trace_id", "")
    if not trace_id:
        return False
    return send_evaluation_to_cloudwatch(
        trace_id=trace_id,
        session_id=case.session_id,
        evaluator_name=evaluator_name,
        score=score,
        explanation=explanation,
        label=label,
        config_id=EVALUATION_CONFIG_ID,
    )


def create_cases_from_session(session, session_id: str, max_cases: int = None) -> List[Case]:
    """Create evaluation cases from a Strands Eval Session."""
    cases = []
    for i, trace in enumerate(session.traces):
        if max_cases and len(cases) >= max_cases:
            break
        agent_span = None
        tool_names = []
        for span in trace.spans:
            if isinstance(span, AgentInvocationSpan):
                agent_span = span
            elif isinstance(span, ToolExecutionSpan):
                tool_names.append(span.tool_call.name)
        if agent_span:
            case = Case(
                name=f"trace_{i+1}_{trace.trace_id[:8]}",
                input=agent_span.user_prompt or "",
                expected_output="",
                session_id=session_id,
                metadata={
                    "actual_output": agent_span.agent_response or "",
                    "actual_trajectory": tool_names,
                    "trace_id": trace.trace_id,
                    "tool_count": len(tool_names),
                },
            )
            cases.append(case)
    return cases

## クライアントの初期化

トレースを取得するための `ObservabilityClient` と変換するための `CloudWatchSessionMapper` を作成します。

マッパーは生の AgentCore Observability スパンを構造化された Strands Eval オブジェクトに変換します:
- trace_id でスパンをグループ化して各インタラクションを再構築
- ツール呼び出しを抽出し、結果と照合
- ユーザープロンプト（最初のメッセージ）とエージェント応答（最終出力）を識別
- AgentInvocationSpan（完全なインタラクション）と ToolExecutionSpan（各ツール使用）を生成

In [ ]:
obs_client = ObservabilityClient(
    region_name=AWS_REGION,
    log_group=SOURCE_LOG_GROUP,
)
mapper = CloudWatchSessionMapper()

end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(hours=LOOKBACK_HOURS)
start_time_ms = int(start_time.timestamp() * 1000)
end_time_ms = int(end_time.timestamp() * 1000)

## セッションの処理

メイン評価ループ。各セッションについて:
1. AgentCore Observability からスパンを取得
2. マッパーを使用してスパンを Strands Eval Session 形式に変換
3. セッション内の各トレースから評価 Cases を作成
4. すべてのケースに対して OutputEvaluator を実行
5. ツールを使用したケースに対して TrajectoryEvaluator を実行
6. ダッシュボード相関用に元のトレース ID でresすべての結果を AgentCore Observability にログ

各セッションの進捗状況が表示されます。エラーはキャッチされてログに記録されますが、ループは停止しません。

In [ ]:
all_session_results = []
total_cases_evaluated = 0
total_logs_sent = 0
all_tools_used = set()

for session_idx, session_info in enumerate(sessions_to_process):
    session_id = session_info.session_id
    print(f"[{session_idx + 1}/{len(sessions_to_process)}] {session_id}")

    try:
        trace_data = obs_client.get_session_data(
            session_id=session_id,
            start_time_ms=start_time_ms,
            end_time_ms=end_time_ms,
            include_runtime_logs=False,
        )

        if not trace_data.spans:
            all_session_results.append({"session_id": session_id, "status": "skipped", "reason": "no_spans"})
            continue

        session = trace_data.to_session(mapper)
        cases = create_cases_from_session(session, session_id, MAX_CASES_PER_SESSION)

        if not cases:
            all_session_results.append({"session_id": session_id, "status": "skipped", "reason": "no_cases"})
            continue

        for case in cases:
            for tool in case.metadata.get("actual_trajectory", []):
                all_tools_used.add(tool)

        # Run Output Evaluator
        output_experiment = Experiment(cases=cases, evaluators=[OutputEvaluator(rubric=output_rubric)])
        output_results = output_experiment.run_evaluations(task_fn)
        output_report = output_results[0]

        output_logged = 0
        for i, case in enumerate(cases):
            if log_case_result_to_cloudwatch(case, OUTPUT_EVALUATOR_NAME, output_report.scores[i], output_report.reasons[i] if i < len(output_report.reasons) else ""):
                output_logged += 1

        # Run Trajectory Evaluator
        trajectory_cases = [c for c in cases if c.metadata.get("actual_trajectory")]
        trajectory_score = None
        trajectory_logged = 0

        if trajectory_cases:
            traj_eval_cases = [
                Case(name=c.name, input=c.input, expected_output=c.expected_output, session_id=c.session_id,
                     metadata={**c.metadata, "trajectory_for_eval": c.metadata.get("actual_trajectory", [])})
                for c in trajectory_cases
            ]
            trajectory_experiment = Experiment(
                cases=traj_eval_cases,
                evaluators=[TrajectoryEvaluator(rubric=trajectory_rubric, trajectory_description={"available_tools": list(all_tools_used)})]
            )
            trajectory_results = trajectory_experiment.run_evaluations(trajectory_task_fn)
            trajectory_report = trajectory_results[0]
            trajectory_score = trajectory_report.overall_score

            for i, case in enumerate(traj_eval_cases):
                if log_case_result_to_cloudwatch(case, TRAJECTORY_EVALUATOR_NAME, trajectory_report.scores[i], trajectory_report.reasons[i] if i < len(trajectory_report.reasons) else ""):
                    trajectory_logged += 1

        all_session_results.append({
            "session_id": session_id,
            "status": "completed",
            "case_count": len(cases),
            "output_score": output_report.overall_score,
            "trajectory_score": trajectory_score,
            "logs_sent": output_logged + trajectory_logged,
        })
        total_cases_evaluated += len(cases)
        total_logs_sent += output_logged + trajectory_logged

    except Exception as e:
        all_session_results.append({"session_id": session_id, "status": "error", "error": str(e)})

print(f"\nCompleted: {len([r for r in all_session_results if r['status'] == 'completed'])} sessions, {total_cases_evaluated} cases, {total_logs_sent} logs sent")

## サマリー

完了率、評価されたケースの総数、出力およびトラジェクトリ評価メトリクスの両方の平均スコアを含む、すべての評価済みセッションの集計統計。

In [ ]:
completed = [r for r in all_session_results if r.get("status") == "completed"]
output_scores = [r["output_score"] for r in completed if r.get("output_score") is not None]
trajectory_scores = [r["trajectory_score"] for r in completed if r.get("trajectory_score") is not None]

print(f"Sessions: {len(completed)}/{len(all_session_results)} completed")
print(f"Cases evaluated: {total_cases_evaluated}")
print(f"CloudWatch logs sent: {total_logs_sent}")

if output_scores:
    print(f"Output score: avg={sum(output_scores)/len(output_scores):.2f}, min={min(output_scores):.2f}, max={max(output_scores):.2f}")
if trajectory_scores:
    print(f"Trajectory score: avg={sum(trajectory_scores)/len(trajectory_scores):.2f}, min={min(trajectory_scores):.2f}, max={max(trajectory_scores):.2f}")

## セッションごとの結果

出力およびトラジェクトリスコアを表示する各セッションの個別結果。「skipped」とマークされたセッションにはスパンまたは有効なケースがありませんでした。「error」とマークされたセッションは処理中に例外が発生しました。

In [ ]:
for i, r in enumerate(all_session_results):
    status = r.get("status", "unknown")
    if status == "completed":
        print(f"{i+1}. {r['session_id'][:20]}... output={r.get('output_score', 0):.2f} traj={r.get('trajectory_score') or '-'}")
    else:
        print(f"{i+1}. {r['session_id'][:20]}... {status}")

## 結果のエクスポート

さらなる分析またはレポート用に評価結果を JSON に保存します。エクスポートには設定、サマリー統計、セッションごとの結果が含まれます。

In [ ]:
import json

export_data = {
    "evaluation_time": datetime.now(timezone.utc).isoformat(),
    "config": {
        "source_log_group": SOURCE_LOG_GROUP,
        "eval_results_log_group": EVAL_RESULTS_LOG_GROUP,
        "output_evaluator": OUTPUT_EVALUATOR_NAME,
        "trajectory_evaluator": TRAJECTORY_EVALUATOR_NAME,
    },
    "summary": {
        "total_sessions": len(all_session_results),
        "completed_sessions": len(completed),
        "total_cases": total_cases_evaluated,
        "avg_output_score": sum(output_scores) / len(output_scores) if output_scores else None,
        "avg_trajectory_score": sum(trajectory_scores) / len(trajectory_scores) if trajectory_scores else None,
    },
    "session_results": all_session_results,
}

with open(RESULTS_JSON_PATH, "w") as f:
    json.dump(export_data, f, indent=2)

print(f"Exported to {RESULTS_JSON_PATH}")